# 🛠️ Utilizarea Avansată a Uneltelor cu Modelele GitHub (.NET)

## 📋 Obiective de Învățare

Acest notebook demonstrează modele de integrare a uneltelor la nivel enterprise folosind Microsoft Agent Framework în .NET cu Modelele GitHub. Vei învăța să construiești agenți sofisticați cu multiple unelte specializate, valorificând tipizarea puternică a C# și caracteristicile enterprise ale .NET.

**Capabilități Avansate ale Uneltelor pe care le vei Stăpâni:**
- 🔧 **Arhitectură Multi-Unelte**: Construirea agenților cu multiple capabilități specializate
- 🎯 **Execuție Tip-Sigură a Uneltelor**: Utilizarea validării la compilare a C#
- 📊 **Modele de Unelte Enterprise**: Design de unelte pregătit pentru producție și gestionarea erorilor
- 🔗 **Compoziția Uneltelor**: Combinarea uneltelor pentru fluxuri de lucru complexe de afaceri

## 🎯 Beneficiile Arhitecturii Uneltelor în .NET

### Caracteristici ale Uneltelor Enterprise
- **Validare la Compilare**: Tipizarea puternică asigură corectitudinea parametrilor uneltelor
- **Injecție de Dependențe**: Integrare cu containerul IoC pentru gestionarea uneltelor
- **Modele Async/Await**: Execuție non-blocantă a uneltelor cu gestionarea corectă a resurselor
- **Jurnalizare Structurată**: Integrare nativă pentru monitorizarea execuției uneltelor

### Modele Pregătite pentru Producție
- **Gestionarea Excepțiilor**: Management complet al erorilor cu excepții tipizate
- **Gestionarea Resurselor**: Modele corecte de eliminare și gestionare a memoriei
- **Monitorizarea Performanței**: Metrici încorporate și contoare de performanță
- **Gestionarea Configurației**: Configurație tip-sigură cu validare

## 🔧 Arhitectura Tehnică

### Componentele de Bază ale Uneltelor în .NET
- **Microsoft.Extensions.AI**: Strat de abstractizare unificat pentru unelte
- **Microsoft.Agents.AI**: Orchestrare de unelte la nivel enterprise
- **Integrarea Modelelor GitHub**: Client API de înaltă performanță cu pooling de conexiuni

### Pipeline-ul de Execuție a Uneltelor
```csharp
User Request → Agent Analysis → Tool Selection → Type Validation
                 ↓               ↓              ↓
         Parameter Binding → Tool Execution → Result Processing → Response
```

## 🛠️ Categorii și Modele de Unelte

### 1. **Unelte de Procesare a Datelor**
- **Validarea Inputului**: Tipizare puternică cu adnotări de date
- **Operațiuni de Transformare**: Conversie și formatare a datelor tip-sigură
- **Logică de Afaceri**: Unelte de calcul și analiză specifice domeniului
- **Formatarea Outputului**: Generarea de răspunsuri structurate

### 2. **Unelte de Integrare** 
- **Conectori API**: Integrare cu servicii RESTful folosind HttpClient
- **Unelte pentru Baze de Date**: Integrare cu Entity Framework pentru acces la date
- **Operațiuni pe Fișiere**: Operațiuni sigure pe sistemul de fișiere cu validare
- **Servicii Externe**: Modele de integrare cu servicii terțe

### 3. **Unelte Utilitare**
- **Procesarea Textului**: Utilitare pentru manipularea și formatarea șirurilor de caractere
- **Operațiuni pe Date/Timp**: Calculații de date/timp sensibile la cultură
- **Unelte Matematice**: Calculații de precizie și operațiuni statistice
- **Unelte de Validare**: Validarea regulilor de afaceri și verificarea datelor

## ⚙️ Cerințe Prealabile și Configurare

**Mediu de Dezvoltare:**
- .NET 9.0 SDK sau mai recent
- Visual Studio 2022 sau VS Code cu extensia C#
- Acces la API-ul Modelelor GitHub

**Pachete NuGet Necesare:**
```xml
<PackageReference Include="Microsoft.Extensions.AI" Version="9.9.0" />
<PackageReference Include="Microsoft.Extensions.AI.OpenAI" Version="9.9.0-preview.1.25458.4" />
<PackageReference Include="DotNetEnv" Version="3.1.1" />
```

**Configurare Mediu (.env file):**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

Ești pregătit să construiești agenți la nivel enterprise cu capabilități puternice și tip-sigure în .NET? Hai să arhitectăm soluții profesionale! 🏢⚡


In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.1"

Installed Packages Microsoft.Extensions.AI, 9.9.1

In [ ]:
#r "nuget: Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.3"

Installed Packages Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.2

In [ ]:
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.251001.3"

Installed Packages microsoft.agents.ai, 1.0.0-preview.251001.2

In [4]:
// #r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

In [5]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [6]:
using System;
using System.ComponentModel;
using System.ClientModel;

using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using OpenAI;

In [7]:
 using DotNetEnv;

In [8]:
Env.Load("../../../.env");

In [9]:
[Description("Provides a random vacation destination.")]
static string GetRandomDestination()
{
    var destinations = new List<string>
    {
        "Paris, France",
        "Tokyo, Japan",
        "New York City, USA",
        "Sydney, Australia",
        "Rome, Italy",
        "Barcelona, Spain",
        "Cape Town, South Africa",
        "Rio de Janeiro, Brazil",
        "Bangkok, Thailand",
        "Vancouver, Canada"
    };

    var random = new Random();
    int index = random.Next(destinations.Count);
    return destinations[index];
}

In [10]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [11]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

In [12]:

var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [13]:
AIAgent agent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    instructions:"You are a helpful AI Agent that can help plan vacations for customers at random destinations", tools: [AIFunctionFactory.Create((Func<string>)GetRandomDestination)]);

In [14]:
AgentThread thread = agent.GetNewThread();

In [15]:
Console.WriteLine(await agent.RunAsync("Plan me a day trip",thread));

Your day trip will be in Paris, France! Here's a proposed itinerary for your day:

### Morning
- **Breakfast at a Local Café**: Start your day with a traditional French breakfast. Try a croissant and café au lait at a local café.
- **Visit the Eiffel Tower**: Arrive early to beat the crowds. Enjoy the stunning views of the city from the top.

### Late Morning
- **Stroll Along the Seine**: Take a leisurely walk along the Seine River and enjoy the picturesque views.
- **Visit Notre-Dame Cathedral**: Explore this iconic Gothic cathedral and its stunning architecture.

### Lunch
- **Lunch at a Bistro**: Treat yourself to a classic French lunch at a nearby bistro. Consider trying coq au vin or a delicious quiche.

### Afternoon
- **Explore the Louvre**: Spend your afternoon at the Louvre Museum. While you may not see everything, be sure to check out the Mona Lisa and other famous artworks.
- **Walk Through the Tuileries Garden**: After the museum, relax in the beautiful Tuileries Garden.

#

In [16]:
Console.WriteLine(await agent.RunAsync("I don't like that destination. Plan me another vacation.",thread));

Your new vacation destination is Rio de Janeiro, Brazil! Here's a proposed itinerary for your trip:

### Day 1: Arrival in Rio
- **Check-in**: Arrive and settle into your accommodation.
- **Evening at Copacabana Beach**: Stroll along the famous Copacabana Beach. Enjoy the vibrant atmosphere and watch the sunset.

### Day 2: Exploring the City
- **Visit Christ the Redeemer**: Start your day with a visit to the iconic Christ the Redeemer statue. Take the train up to the top for breathtaking views.
- **Explore Santa Teresa**: Wander through the charming streets of Santa Teresa, known for its colonial-style houses, art studios, and local shops.
- **Lunch at a Local Restaurant**: Enjoy traditional Brazilian cuisine, such as feijoada (black bean stew).
- **Afternoon at Sugarloaf Mountain**: Take a cable car ride to Sugarloaf Mountain for panoramic views of the city and coastline.

### Day 3: Cultural Experience
- **Visit the Selarón Steps**: Walk up the colorful Selarón Steps, an iconic mosa


---

**Declinarea responsabilității**:  
Acest document a fost tradus utilizând serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși depunem eforturi pentru a asigura acuratețea, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa nativă trebuie considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist. Nu ne asumăm răspunderea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
